In [4]:
#https://www.kaggle.com/anikannal/solar-power-generation-data

In [9]:
import pandas as pd
import numpy as np
%matplotlib inline

In [10]:
full = pd.read_parquet("full_plant1.parquet").sort_values("DATE_TIME")

In [11]:
full.head()

DATE_TIME  PLANT_ID       SOURCE_KEY  DC_POWER  AC_POWER  DAILY_YIELD  \
0  2020-05-15   4135001  1BY6WEcLGh8j5v7       0.0       0.0          0.0   
20 2020-05-15   4135001  zVJPv84UY57bAof       0.0       0.0          0.0   
19 2020-05-15   4135001  zBIq5rxdHJRwDNY       0.0       0.0          0.0   
18 2020-05-15   4135001  z9Y9gH1T5YWrNuG       0.0       0.0          0.0   
17 2020-05-15   4135001  wCURE6d3bPkepu2       0.0       0.0          0.0   

    TOTAL_YIELD  AMBIENT_TEMPERATURE  MODULE_TEMPERATURE  IRRADIATION  
0     6259559.0            25.184316           22.857507          0.0  
20    7116151.0            25.184316           22.857507          0.0  
19    6339380.0            25.184316           22.857507          0.0  
18    7007866.0            25.184316           22.857507          0.0  
17    6782598.0            25.184316           22.857507          0.0

# Recursive

In [33]:
full_recursive = []

for source_key, source_key_df in full.groupby('SOURCE_KEY'):
    source_key_df = source_key_df.copy()
    source_key_df['15M_YIELD'] = source_key_df['DAILY_YIELD'].diff() # t - t-1
    
    #mostrar antes da trafo
    first_data_points_of_day = (source_key_df['DATE_TIME'].dt.hour == 0) & (source_key_df['DATE_TIME'].dt.minute <= 15)
    source_key_df.loc[first_data_points_of_day, '15M_YIELD'] = 0.
    
    record_number = source_key_df.groupby(source_key_df['DATE_TIME'].dt.date)['DATE_TIME'].rank()
    #print(record_number)
    source_key_df.loc[record_number <= 1, '15M_YIELD'] = 0
    #source_key_df['15M_YIELD'].plot()
    
    source_key_df['Y'] = source_key_df['15M_YIELD'].shift(-1)
    source_key_df = source_key_df.iloc[:-1]
    
    full_recursive.append(source_key_df)
    
full_recursive_df = pd.concat(full_recursive, axis=0, ignore_index=True)

# Direct
- pode usar features com atraso

In [36]:
full_direct = []
lead_t = 4

for source_key, source_key_df in full.groupby('SOURCE_KEY'):
    source_key_df = source_key_df.copy()
    source_key_df['15M_YIELD'] = source_key_df['DAILY_YIELD'].diff() # t - t-1
    
    first_data_points_of_day = (source_key_df['DATE_TIME'].dt.hour == 0) & (source_key_df['DATE_TIME'].dt.minute <= 15)
    source_key_df.loc[first_data_points_of_day, '15M_YIELD'] = 0.
    
    record_number = source_key_df.groupby(source_key_df['DATE_TIME'].dt.date)['DATE_TIME'].rank()
    #print(record_number)
    source_key_df.loc[record_number <= 1, '15M_YIELD'] = 0
    #source_key_df['15M_YIELD'].plot()
    
    source_key_df['Y{}'.format(lead_t)] = source_key_df['15M_YIELD'].shift(-lead_t)
    source_key_df = source_key_df.iloc[:-lead_t]
    
    full_direct.append(source_key_df)
    
    #break
full_direct_df = pd.concat(full_direct, axis=0, ignore_index=True)

# Native
- pode usar features com atraso
- multioutput prediction

In [39]:
full_native = []
min_lead_t = 1
max_lead_t = 4


for source_key, source_key_df in full.groupby('SOURCE_KEY'):
    source_key_df = source_key_df.copy()
    source_key_df['15M_YIELD'] = source_key_df['DAILY_YIELD'].diff() # t - t-1

    first_data_points_of_day = (source_key_df['DATE_TIME'].dt.hour == 0) & (source_key_df['DATE_TIME'].dt.minute <= 15)
    source_key_df.loc[first_data_points_of_day, '15M_YIELD'] = 0.
    
    record_number = source_key_df.groupby(source_key_df['DATE_TIME'].dt.date)['DATE_TIME'].rank()
    #print(record_number)
    source_key_df.loc[record_number <= 1, '15M_YIELD'] = 0
    #source_key_df['15M_YIELD'].plot()
    
    for lead_t in range(min_lead_t, max_lead_t+1):
        source_key_df['Y{}'.format(lead_t)] = source_key_df['15M_YIELD'].shift(-lead_t)
        source_key_df = source_key_df.iloc[:-lead_t]
    
    full_native.append(source_key_df)
    
    #break
full_native_df = pd.concat(full_native, axis=0, ignore_index=True)

In [40]:
full_native_df.head()

DATE_TIME  PLANT_ID       SOURCE_KEY  DC_POWER  AC_POWER  \
0 2020-05-15 00:00:00   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
1 2020-05-15 00:15:00   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
2 2020-05-15 00:30:00   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
3 2020-05-15 00:45:00   4135001  1BY6WEcLGh8j5v7       0.0       0.0   
4 2020-05-15 01:00:00   4135001  1BY6WEcLGh8j5v7       0.0       0.0   

   DAILY_YIELD  TOTAL_YIELD  AMBIENT_TEMPERATURE  MODULE_TEMPERATURE  \
0          0.0    6259559.0            25.184316           22.857507   
1          0.0    6259559.0            25.084589           22.761668   
2          0.0    6259559.0            24.935753           22.592306   
3          0.0    6259559.0            24.846130           22.360852   
4          0.0    6259559.0            24.621525           22.165423   

   IRRADIATION  15M_YIELD   Y1   Y2   Y3   Y4  
0          0.0        0.0  0.0  0.0  0.0  0.0  
1          0.0        0.0  0.0  0.0  0.0  0.0  
2          0.0        0.0  0.0  0.0  0.0  0.0  
3          0.0        0.0  0.0  0.0  0.0  0.0  
4          0.0        0.0  0.0  0.0  0.0  0.0

# Qual vamos usar?

In [41]:
full_direct_df.to_parquet("full_direct_df.parquet")